# More Pulling Together

In this lecture we'll just work on our example a bit more and pull together some of the concepts we've learned in the previous lectures.

First we need to define our `Forest` and `BurnableForest` classes used in previous lectures.

In [ ]:
import numpy as np
from matplotlib.pyplot import plot, legend
%matplotlib inline

class Forest(object):
    """ A Forest can grow but won't burn.
    """

    def __init__(self, size=(150, 150), p_sapling=0.0025):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def _rand_bool(self, p):
        return np.random.uniform(size=self.trees.shape) < p

    def advance_one_step(self):
        self.grow_trees()


class BurnableForest(Forest):
    """ BurnableForest supports fires.
    """

    def __init__(self, p_lightning=5.0e-6, *args, **kwargs):
        super(BurnableForest, self).__init__(*args, **kwargs)
        self.p_lightning = p_lightning
        self.fires = np.zeros(self.size, dtype=bool)

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

    def burn_trees(self):
        """The rules for burning trees:
        -New fires start in tree-filled adjacent cells.
        -Trees in currently burning cells burn and disappear.

        The fire-spreading algorithm is calculated on an array padded
        around the edges with zeros to allow vectorized computations.
        """
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

Now we'll subclass `BurnableForest` and add more methods to implement our original burning behavior.  This code is pretty much copied and pasted directly from our earlier implementation.

In [ ]:
class SlowBurnForest(BurnableForest):
    """BurnableForest where fires spread at the same time scale as the new
    trees grow.
    """
    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

    def burn_trees(self):
        # assumes 2D forest
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

Note that in `SlowBurnForest` we completely overwrote `advance_one_step()`.  The method has the same name, but the code is unrelated to the parent class.

We have the original behavior we started with:

In [ ]:
forest = SlowBurnForest()
history = []
for i in range(1000):
    forest.advance_one_step()
    history.append(forest.tree_fraction)
plot(history)

Notice that I was able to place a `SlowBurnForest` instance into exactly the same code that I used earlier for the `Forest` instance.

Now I can take my co-worker's suggestions and re-use the `BurnableForest` code in a different class, called InstantBurnForest.  Again, I'm completely replacing the `advance_one_step()` method.

Notice that all of the functionality that is shared between the `SlowBurnForest` and the `InstantBurnForest` is written in `BurnableForest`.

In [ ]:
from scipy.ndimage.measurements import label
class InstantBurnForest(BurnableForest):
    """BurnableForest where fires spread instantaneously relative to the
    growth of new saplings.
    """
    def advance_one_step(self):
        self.grow_trees()
        self.strike_and_burn()

    def strike_and_burn(self):
        strikes = self._rand_bool(self.p_lightning) & self.trees
        groves, num_groves = label(self.trees)
        fires = set(groves[strikes])
        self.fires.fill(False)
        for fire in fires:
            self.fires[groves == fire] = True
        self.trees[self.fires] = False

In [ ]:
forest = InstantBurnForest()
history = []
for i in range(1000):
    forest.advance_one_step()
    history.append(forest.tree_fraction)
plot(history)

Now there are catastrophic fires.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com